In [1]:
import numpy, matplotlib, time, copy, random
directory = '../data/'
file_paths = ['a_example.txt', 'b_read_on.txt', 'c_incunabula.txt','d_tough_choices.txt', 'e_so_many_books.txt', 'f_libraries_of_the_world.txt']

In [2]:
class Library():
    def __init__(self,index,N,T,M, book_ids: set):
        self.id = index
        self.size  = N
        self.signup_time = T
        self.books_per_day = M
        self.book_ids = book_ids
        
    def get_n_best_books(self,n, book_values):
        result = [];
        k=0
        for i in range(len(self.book_ids)):
            if book_values[self.book_ids[i]] != 0:
                result.append(self.book_ids[i])
                k+=1
                if k == n:
                    return tuple(result)
        return tuple(result)
    
    def __str__(self):
        return str(self.__class__) + ": " + str(self.__dict__)
        
        

def process_file(filePath):
    with open(directory+filePath,'r') as file:
        content = file.read().split('\n')[:-1];
        B,L,D = content[0].split()
        B,L,D = int(B),int(L),int(D)
    bookValues = [int(n) for n in content[1].split()]#tuple([int(n) for n in content[1].split()])
    libraries = []
    def book_sort(book):
        return bookValues[book]
    for i in range(L):
        N,T,M = content[2+2*i].split()
        N,T,M=int(N),int(T),int(M)
        book_ids = tuple(sorted([int(id) for id in content[2+2*i+1].split()], key = book_sort, reverse=True))
        libraries.append(Library(i,N,T,M,book_ids))
    return ((B,L,D), bookValues, libraries)

In [3]:
def check_solution(D, libraries):
    days = 0
    prev_books = set()
    for library in libraries:
        days+=library.signup_time
        if len(library.book_ids) > (D-days)*library.books_per_day:
            print("what", len(library.book_ids), (D-days)*library.books_per_day)
        assert(not any([(book in prev_books) for book in library.book_ids]))
        prev_books.update(library.book_ids)
    assert(days < D)

In [7]:
def get_best_solution(file_path):
    (B,L,D),book_values, libraries = process_file(file_path)
    
    def get_score(selected_lib_ids):
        local_book_values = copy.copy(book_values)
        day = 0
        def book_sort(book):
            return local_book_values[book]
        def library_score(lib):
            delta_time = D - lib.signup_time
            n_best_books = lib.get_n_best_books(delta_time*lib.signup_time, local_book_values)
            sum_of_best_book_scores = sum([local_book_values[book] for book in n_best_books])
            return sum_of_best_book_scores/lib.signup_time
        sorted_libraries = sorted([copy.copy(libraries[i]) for i in selected_lib_ids],key=library_score, reverse=True)
        it=0;
        while it < len(sorted_libraries):
            next_library = sorted_libraries[it]
            day+=next_library.signup_time
            if day>=D:
                break
            next_library.book_ids = next_library.get_n_best_books((D-day)*next_library.books_per_day, local_book_values)
            for book_id in next_library.book_ids:
                local_book_values[book_id] = 0
            it+=1
        score = 0
        for i in range(0,it):
            for idk in sorted_libraries[i].book_ids:
                score+=book_values[idk]    
        check_solution(D, sorted_libraries[:it])
        print("Scoer ", score)
        return score
    
    max_score = get_score([i for i in range(len(libraries))])
    return max_score
    
sum_scores = 0
for file_path in file_paths:
    print(file_path)
    %time sum_scores +=get_best_solution(file_path)
print("SUM SCORES", sum_scores)

a_example.txt
Scoer  21
CPU times: user 581 µs, sys: 74 µs, total: 655 µs
Wall time: 505 µs
b_read_on.txt
Scoer  5822900
CPU times: user 72.8 ms, sys: 73 µs, total: 72.9 ms
Wall time: 72.4 ms
c_incunabula.txt
Scoer  5645747
CPU times: user 145 ms, sys: 0 ns, total: 145 ms
Wall time: 145 ms
d_tough_choices.txt
Scoer  4815395
CPU times: user 384 ms, sys: 0 ns, total: 384 ms
Wall time: 383 ms
e_so_many_books.txt
Scoer  4613373
CPU times: user 266 ms, sys: 0 ns, total: 266 ms
Wall time: 265 ms
f_libraries_of_the_world.txt
Scoer  5240161
CPU times: user 311 ms, sys: 3.9 ms, total: 314 ms
Wall time: 314 ms
SUM SCORES 26137597
